In [57]:
import sys 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv), data manipulation as in SQL


## for Deep-learing:
import keras
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dropout
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization

import time
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import boto3
import s3fs
import sagemaker
from sagemaker import get_execution_role

# Set up the environment

In [58]:
import os
import sagemaker
from sagemaker import get_execution_role

# initiate session
sagemaker_session = sagemaker.Session()

#define specific AWS Role
role = 'arn:aws:iam::' 

# Upload the data to a S3 bucket

In [62]:
inputs = sagemaker_session.upload_data(path='data', key_prefix='data/TF_Test')

model_artifacts_location = 's3://data/TF_Test/artifacts'

In [4]:
np.random.seed(1)

In [55]:
# reveal referenced .py file needed for training
!cat 'KERAS.py'

import numpy as np
import os
import tensorflow as tf
from tensorflow.python.keras.layers import InputLayer, Conv2D, Activation, MaxPooling2D, Dropout, Flatten, Dense, LSTM
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.saved_model.signature_constants import PREDICT_INPUTS

from tensorflow.python.estimator.export.export import build_raw_serving_input_receiver_fn
from tensorflow.python.estimator.export.export_output import PredictOutput


INPUT_TENSOR_NAME = "PREDICT_INPUTS_input" # needs to match the name of the first layer + "_input"


def keras_model_fn(hyperparameters):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.LSTM(256, return_sequences=True,input_shape=(1,16), activation='relu', name='PREDICT_INPUTS'))
    model.add(tf.keras.layers.Dropout(0.9))
    model.add(tf.keras.layers.LSTM(256))
    model.add(tf.keras.layers.Dropout(0.9))
    model.add(tf.keras.layers.Dense(1,act

# Submitting script for training

In [63]:
from sagemaker.tensorflow import TensorFlow

TF_estimator = TensorFlow(entry_point='KERAS.py',
                               role=role,
                               output_path=model_artifacts_location,
                               framework_version='1.10.0',
                               training_steps= 100,                                  
                               evaluation_steps= 100,
                               hyperparameters={'learning_rate': 0.001},
                               train_instance_count = 1,
                               train_instance_type='ml.c4.4xlarge')

TF_estimator.fit(inputs)

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-786739926626
INFO:sagemaker:Creating training-job with name: sagemaker-tensorflow-2018-11-30-00-06-57-506


2018-11-30 00:06:57 Starting - Starting the training job...
2018-11-30 00:06:59 Starting - Launching requested ML instances......
2018-11-30 00:08:04 Starting - Preparing the instances for training......
2018-11-30 00:09:08 Downloading - Downloading input data...
2018-11-30 00:09:55 Training - Downloading the training image..
2018-11-30 00:09:58,997 INFO - root - running container entrypoint
2018-11-30 00:09:58,997 INFO - root - starting train task
2018-11-30 00:09:59,002 INFO - container_support.training - Training starting
2018-11-30 00:10:03,995 INFO - tf_container - ----------------------TF_CONFIG--------------------------
2018-11-30 00:10:03,995 INFO - tf_container - {"environment": "cloud", "cluster": {"master": ["algo-1:2222"]}, "task": {"index": 0, "type": "master"}}
2018-11-30 00:10:03,995 INFO - tf_container - ---------------------------------------------------------
2018-11-30 00:10:03,995 INFO - tf_container - creating RunConfig:
2018-11-30 00:10:03,995 INFO - tf_container 


2018-11-30 00:10:07 Training - Training image download completed. Training in progress.2018-11-30 00:10:14,474 WARNING - tensorflow - From /opt/ml/code/KERAS.py:46: load_csv_without_header (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.data instead.
2018-11-30 00:10:23,460 INFO - tensorflow - Calling model_fn.
2018-11-30 00:10:25,069 INFO - tensorflow - Done calling model_fn.
2018-11-30 00:10:25,070 INFO - tensorflow - Create CheckpointSaverHook.
2018-11-30 00:10:25.071088: I tensorflow/core/platform/s3/aws_logging.cc:54] Connection has been released. Continuing.
2018-11-30 00:10:25.107641: E tensorflow/core/platform/s3/aws_logging.cc:60] No response body. Response code: 404
2018-11-30 00:10:25.107691: W tensorflow/core/platform/s3/aws_logging.cc:57] If the signature check failed. This could be because of a time skew. Attempting to adjust the signer.
2018-11-30 00:10:25.107799: I tenso

2018-11-30 00:10:36,737 INFO - tensorflow - Saving checkpoints for 100 into s3://sagemaker-us-east-1-786739926626/data/TF_Test/artifacts/sagemaker-tensorflow-2018-11-30-00-06-57-506/checkpoints/model.ckpt.
2018-11-30 00:10:36.738081: I tensorflow/core/platform/s3/aws_logging.cc:54] Connection has been released. Continuing.
2018-11-30 00:10:36.757279: I tensorflow/core/platform/s3/aws_logging.cc:54] Deleting file: /tmp/s3_filesystem_XXXXXX20181130T0010361543536636737
2018-11-30 00:10:36.765571: I tensorflow/core/platform/s3/aws_logging.cc:54] Connection has been released. Continuing.
2018-11-30 00:10:37.138736: I tensorflow/core/platform/s3/aws_logging.cc:54] Deleting file: /tmp/s3_filesystem_XXXXXX20181130T0010361543536636757
2018-11-30 00:10:37.140205: I tensorflow/core/platform/s3/aws_logging.cc:54] Connection has been released. Continuing.
2018-11-30 00:10:37.160657: I tensorflow/core/platform/s3/aws_logging.cc:54] Connection has been released. Continuing.
2018-11-30 00:10:37.465125

2018-11-30 00:10:54.735892: I tensorflow/core/platform/s3/aws_logging.cc:54] Connection has been released. Continuing.
2018-11-30 00:10:54.747728: I tensorflow/core/platform/s3/aws_logging.cc:54] Connection has been released. Continuing.
2018-11-30 00:10:54.772890: I tensorflow/core/platform/s3/aws_logging.cc:54] Deleting file: /tmp/s3_filesystem_XXXXXX20181130T0010541543536654746
2018-11-30 00:10:54.773054: I tensorflow/core/platform/s3/aws_logging.cc:54] Connection has been released. Continuing.
2018-11-30 00:10:54.796006: I tensorflow/core/platform/s3/aws_logging.cc:54] Connection has been released. Continuing.
2018-11-30 00:10:54.816746: I tensorflow/core/platform/s3/aws_logging.cc:54] Connection has been released. Continuing.
2018-11-30 00:10:54.938367: I tensorflow/core/platform/s3/aws_logging.cc:54] Connection has been released. Continuing.
2018-11-30 00:10:54.947743: E tensorflow/core/platform/s3/aws_logging.cc:60] No response body. Response code: 404
2018-11-30 00:10:54.947771

# Submiting a trained model for hosting

The deploy() method creates an endpoint which serves prediction requests in real-time.

In [101]:
TF_predictor = TF_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.4xlarge')

INFO:sagemaker:Creating model with name: sagemaker-tensorflow-2018-11-30-00-06-57-506
INFO:sagemaker:Creating endpoint with name sagemaker-tensorflow-2018-11-30-00-06-57-506


---------------------------------------------------------------!

In [128]:
import tensorflow as tf
import numpy as np

prediction_set = tf.contrib.learn.datasets.base.load_csv_without_header(
    filename=os.path.join('data/Test_Clean.csv'), target_dtype=np.float32, features_dtype=np.float32)

In [161]:
data = np.array(prediction_set[0][0]).reshape((-1,1,16))

In [162]:
data.shape

(1, 1, 16)

In [163]:
TF_predictor.predict({'PREDICT_INPUTS_input':data})

{'outputs': {'dense': {'dtype': 1,
   'tensor_shape': {'dim': [{'size': 1}, {'size': 1}]},
   'float_val': [0.12401328980922699]}},
 'model_spec': {'name': 'generic_model',
  'version': {'value': 1543536652},
  'signature_name': 'serving_default'}}

In [164]:
sagemaker.Session().delete_endpoint(TF_predictor.endpoint)

INFO:sagemaker:Deleting endpoint with name: \n